In [1]:
import pickle
import pandas as pd
from tensorflow.keras.layers import Dense,Input,Conv2D,MaxPooling2D,Flatten
from tensorflow.keras.models import Model
import numpy as np
from sklearn.metrics import classification_report

In [2]:
pickle_loc=('preprocessed_files.pkl')
data=pd.read_pickle(pickle_loc)


In [3]:
data.keys()

dict_keys(['X_train', 'X_test', 'y_smile_train', 'y_smile_test', 'y_hat_train', 'y_hat_test'])

In [4]:
X_train=data['X_train']
X_test=data['X_test']
y_smile_train=data['y_smile_train']
y_smile_test=data['y_smile_test']
y_hat_train=data['y_hat_train']
y_hat_test=data['y_hat_test']

In [5]:
X_train.shape

(172209, 64, 64, 3)

In [6]:
y_hat_train.shape

(172209,)

In [7]:
input_layer=Input(shape=(64,64,3),name='input_layer')
conv1=Conv2D(32,(3,3),activation='relu')(input_layer)
max1=MaxPooling2D((2,2))(conv1)
conv2=Conv2D(64,(3,3),activation='relu')(max1)
max2=MaxPooling2D((2,2))(conv2)
conv3=Conv2D(64,(3,3),activation='relu')(max2)
flatten=Flatten()(conv3)
dense=Dense(256,activation='relu')(flatten)

In [8]:
smile_dense=Dense(256,activation='relu')(dense)
smile=Dense(1, activation='sigmoid',name='smile')(smile_dense)

In [9]:
hat_dense=Dense(256,activation='relu')(dense)
hat=Dense(1, activation='sigmoid',name='hat')(hat_dense)

In [10]:
model=Model(inputs=input_layer,outputs=[smile,hat])

In [11]:
model.compile(optimizer='adam',loss='binary_crossentropy', metrics={'smile':'accuracy','hat':'accuracy'})

In [12]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 64, 64, 3) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 62, 62,    │        896 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 31, 31,    │          0 │ conv2d[0][0]      │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 29, 29,    │     18,496 │ max_pooling2d[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 14, 14,    │          0 │ conv2d_1[0][0]    │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 12, 12,    │     36,928 │ max_pooling2d_1[… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 9216)      │          0 │ conv2d_2[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 256)       │  2,359,552 │ flatten[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 256)       │     65,792 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 256)       │     65,792 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ smile (Dense)       │ (None, 1)         │        257 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ hat (Dense)         │ (None, 1)         │        257 │ dense_2[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 2,547,970 (9.72 MB)

 Trainable params: 2,547,970 (9.72 MB)

 Non-trainable params: 0 (0.00 B)

In [13]:
model.fit(X_train,[y_smile_train,y_hat_train],epochs=2,batch_size=32)

Epoch 1/2
5382/5382 ━━━━━━━━━━━━━━━━━━━━ 93s 17ms/step - hat_accuracy: 0.9737 - hat_loss: 0.0777 - loss: 0.3682 - smile_accuracy: 0.8649 - smile_loss: 0.2905
Epoch 2/2
5382/5382 ━━━━━━━━━━━━━━━━━━━━ 91s 17ms/step - hat_accuracy: 0.9838 - hat_loss: 0.0464 - loss: 0.2437 - smile_accuracy: 0.9171 - smile_loss: 0.1973


In [14]:
results = model.evaluate(X_test,[y_smile_test,y_hat_test])

950/950 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - hat_accuracy: 0.9843 - hat_loss: 0.0485 - loss: 0.2330 - smile_accuracy: 0.9215 - smile_loss: 0.1844


In [15]:
X_test.shape

(30390, 64, 64, 3)

In [48]:
prediction=model.predict(X_test)

950/950 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step


In [49]:
np.array(prediction).shape

(2, 30390, 1)

In [50]:
prediction=np.array(prediction)

In [51]:
prediction[1]

array([[2.5277650e-05],
       [1.4833396e-04],
       [1.8365292e-08],
       ...,
       [6.6799541e-05],
       [9.4666888e-09],
       [6.9147598e-07]], dtype=float32)

In [52]:
prediction2=prediction[:,:,0]

In [57]:
prediction2=prediction2.transpose()

In [60]:
predictions_df = pd.DataFrame(columns=['smile','hat'], data=prediction2)
predictions_df = round(predictions_df,0)
predictions_df

,smile,hat
0,0.0,0.0
1,0.0,0.0
2,1.0,0.0
3,0.0,0.0
4,0.0,0.0
...,...,...
30385,0.0,0.0
30386,1.0,0.0
30387,1.0,0.0
30388,1.0,0.0


In [26]:
y_smile_test

array([0, 0, 1, ..., 1, 1, 0], dtype=int64)

In [63]:
print(classification_report(y_hat_test,predictions_df['hat']))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99     29065
           1       0.89      0.77      0.83      1325

    accuracy                           0.99     30390
   macro avg       0.94      0.88      0.91     30390
weighted avg       0.99      0.99      0.99     30390



In [64]:
print(classification_report(y_smile_test,predictions_df['smile']))

              precision    recall  f1-score   support

           0       0.92      0.92      0.92     15380
           1       0.92      0.92      0.92     15010

    accuracy                           0.92     30390
   macro avg       0.92      0.92      0.92     30390
weighted avg       0.92      0.92      0.92     30390



In [65]:
model.save('faces.keras')